In [15]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [16]:
import os
import pickle
import pandas as pd
import financial.data as fd
import financial.momentum.auxiliarFunctions as auxF
from financial.momentum.exponentialRegression import ExponentialRegressionModelFactory
from financial.io.file.cache import FileCache

# Configuración de prueba
ticker = "AAPL"
cache_path=os.environ["CACHE"]+"/"  # Ruta donde se guardarán los datos
model_name = "exponential"

# Crear el DataStore
ds = fd.CachedDataStore(path=os.environ.get("DATA", "./data"), cache=FileCache(cache_path=cache_path))

def local_regression_features_wrapper(ds: fd.DataStore) -> fd.Set:
    '''
    Wrapper function to generate local regression features.
    This is required by ModelFactory when creating a model.
    '''
    return auxF.local_regression_features(ds, ticker)

# Ejecutar la función para almacenar los datos
print(f"⚙️  Almacenando datos de momentum para {ticker}...")
auxF.store_exponentialModel_data(ticker, ds, cache_path)



⚙️  Almacenando datos de momentum para AAPL...


ValueError: Length of values (8727) does not match length of index (8797)

In [ ]:
# Intentar recuperar los archivos guardados

momentum_path = os.path.join(cache_path, f"model/momentum/{model_name}/{ticker}.pkl")
r2_path = os.path.join(cache_path, f"model/momentum/{model_name}/{ticker}@r2.pkl")

# Comprobar si los archivos existen
if os.path.exists(momentum_path) and os.path.exists(r2_path):
    print(f"✅ Archivos encontrados: {momentum_path} y {r2_path}")
    
    # Cargar los archivos y comprobar los datos
    with open(momentum_path, 'rb') as file:
        momentum_series = pickle.load(file)
        print("\n📊 **Momentum Series (Beta) - Últimos valores guardados:**")
        print(momentum_series.apply(lambda x: f"{x:.15f}"))


    with open(r2_path, 'rb') as file:
        r2_series = pickle.load(file)
        print("\n📊 **R² Series - Últimos valores guardados:**")
        print(r2_series)

else:
    print("❌ ERROR: No se encontraron los archivos de caché.")